# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-22-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-22-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-23 05:23:41,33.93911,67.709953,55617,2433,48895,4289.0,Afghanistan,142.870203,4.374562
1,NaN,NaN,NaN,Albania,2021-02-23 05:23:41,41.15330,20.168300,101285,1681,64318,35286.0,Albania,3519.528807,1.659673
2,NaN,NaN,NaN,Algeria,2021-02-23 05:23:41,28.03390,1.659600,112094,2964,77225,31905.0,Algeria,255.624479,2.644209
3,NaN,NaN,NaN,Andorra,2021-02-23 05:23:41,42.50630,1.521800,10712,109,10245,358.0,Andorra,13863.974633,1.017550
4,NaN,NaN,NaN,Angola,2021-02-23 05:23:41,-11.20270,17.873900,20548,499,19190,859.0,Angola,62.520028,2.428460


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55473,55492,55514,55518,55540,55557,55575,55580,55604,55617
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,91987,93075,93850,94651,95726,96838,97909,99062,100246,101285
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,110513,110711,110894,111069,111247,111418,111600,111764,111917,112094


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2427,2427,2427,2428,2428,2430,2430,2430,2432,2433
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1543,1555,1567,1582,1600,1617,1636,1653,1666,1681
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2935,2939,2943,2945,2947,2950,2954,2958,2961,2964


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48392,48395,48433,48514,48626,48798,48803,48820,48834,48895
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,55983,56764,57736,58794,59684,60675,61605,62533,63329,64318
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,75816,75999,76168,76330,76489,76640,76797,76940,77076,77225


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6002,6023,6024,6038,6050,6071,6079,6092,6117,6121
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19051,19105,19136,19176,19267,19324,19361,19392,19433,19461
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2040,2042,2044,2055,2053,2057,2061,2067,2070,2074


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,84,84,84,84,84,84,85,85,85,85
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,252,252,252,253,253,254,262,262,262,262
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,48,48,48,48,49,49,49,50,50,50


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-02-23 05:23:41,32.539527,-86.644082,6121,85,0,6036.0,"Autauga, Alabama, US",10955.986325,1.388662
689,1075.0,Lamar,Alabama,US,2021-02-23 05:23:41,33.779950,-88.096680,1296,32,0,1264.0,"Lamar, Alabama, US",9387.902934,2.469136
690,1077.0,Lauderdale,Alabama,US,2021-02-23 05:23:41,34.901719,-87.656247,8525,208,0,8317.0,"Lauderdale, Alabama, US",9193.456200,2.439883


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,28188571,500244,0
India,11016434,156463,10712665
Brazil,10195160,247143,9108205


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,28188571,500244,0,27688327,2021-02-23 05:23:41,37.936303,-91.379001
India,11016434,156463,10712665,147306,2021-02-23 05:23:41,23.088275,81.806127
Brazil,10195160,247143,9108205,839812,2021-02-23 05:23:41,-12.669522,-48.480493
United Kingdom,4138233,120988,11342,4005903,2021-02-23 05:23:41,30.744028,-35.367255
Russia,4130447,82255,3684955,363237,2021-02-23 05:23:41,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3535358,49475,0
Texas,2606513,42301,0
Florida,1872923,30065,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3535358,49475,0,3485883,2021-02-23 05:23:41,37.843962,-120.728594
Texas,2606513,42301,0,2564212,2021-02-23 05:23:41,31.660643,-98.653069
Florida,1872923,30065,0,1842858,2021-02-23 05:23:41,28.940755,-82.700744
New York,1597969,46924,0,1551045,2021-02-23 05:23:41,42.544151,-75.474183
Illinois,1175568,22506,0,1153062,2021-02-23 05:23:41,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1181403,19904,0
Arizona,Maricopa,506046,8833,0
Illinois,Cook,470391,9278,0
Florida,Miami-Dade,403259,5314,0
Texas,Harris,343070,4858,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1181403,19904,0,1161499,2021-02-23 05:23:41,34.308284,-118.228241,6037.0
Arizona,Maricopa,506046,8833,0,497213,2021-02-23 05:23:41,33.348359,-112.491815,4013.0
Illinois,Cook,470391,9278,0,461113,2021-02-23 05:23:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,403259,5314,0,397945,2021-02-23 05:23:41,25.611236,-80.551706,12086.0
Texas,Harris,343070,4858,0,338212,2021-02-23 05:23:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,55580,99062,111764,10672,20499,598,2060625,170234,28926,443536,...,4117739,52163,79632,1,135603,2368,172315,2157,73894,35768
2021-02-21,55604,100246,111917,10699,20519,598,2064334,170402,28930,445374,...,4127574,52815,79654,1,136068,2383,173635,2165,74503,35796
2021-02-22,55617,101285,112094,10712,20548,614,2069751,170506,28937,446644,...,4138233,53310,79681,1,136545,2392,174969,2176,75027,35862


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,2430,1653,2958,107,498,11,51122,3159,909,8368,...,120593,569,622,0,1312,35,1971,618,1016,1432
2021-02-21,2432,1666,2961,107,499,13,51198,3164,909,8386,...,120810,574,622,0,1316,35,1976,619,1020,1436
2021-02-22,2433,1681,2964,109,499,14,51359,3167,909,8397,...,120988,583,622,0,1320,35,1986,619,1031,1441


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,48820,62533,76940,10170,19005,217,1861433,161903,22881,419283,...,11293,45881,78127,1,127598,1627,160763,1432,66943,32096
2021-02-21,48834,63329,77076,10206,19013,218,1866501,161994,22882,420509,...,11311,46265,78152,1,128088,1702,161410,1432,67944,32125
2021-02-22,48895,64318,77225,10245,19190,222,1872213,162098,22885,421811,...,11342,46845,78171,1,128579,1717,162025,1432,68928,32216


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6002,6023,6024,6038,6050,6071,6079,6092,6117,6121
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1277,1281,1281,1283,1282,1286,1291,1292,1294,1296
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8460,8471,8476,8479,8480,8508,8513,8515,8523,8525
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2771,2777,2780,2781,2781,2787,2790,2794,2795,2798
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14563,14608,14622,14641,14684,14729,14766,14779,14798,14809


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,485986,57393,0,806163,315230,3527338,419812,273101,84531,49,...,764008,2593816,366034,14359,2575,561812,333794,129364,611789,53683
2021-02-21,486843,57393,0,807967,315514,3531846,420614,273101,84732,49,...,765137,2600660,366735,14493,2575,564115,333794,129616,612240,53795
2021-02-22,487520,57738,0,809474,315759,3535358,421294,275334,85090,49,...,766089,2606513,367073,14608,2579,565270,334962,129854,612712,53900


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-20        6092   19392    2067  2414   6040    1149   1938   12915   
2021-02-21        6117   19433    2070  2416   6042    1151   1940   12940   
2021-02-22        6121   19461    2074  2417   6043    1153   1945   13017   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-20         3364     1753  ...    2590    580     2988      671   
2021-02-21         3367     1753  ...    2597    580     2992      671   
2021-02-22         3367     1753  ...    2603    583     2997      671   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-20           3645  3318  2024          0      877    619  
2021-02-21           3660  3326  2027          0      877    619  
2021-02-22           3686  3331  2030          0      879    620  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,9590,289,0,15480,5348,49120,5887,7523,1366,0,...,11115,42041,1842,196,25,7197,4822,2254,6871,662
2021-02-21,9592,289,0,15505,5357,49345,5892,7523,1367,0,...,11133,42162,1852,197,25,7331,4822,2261,6871,662
2021-02-22,9592,290,0,15502,5363,49475,5893,7562,1368,0,...,11153,42301,1853,198,25,7486,4857,2263,6871,662


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-20          85     262      50   58    125      33     64     278   
2021-02-21          85     262      50   58    125      33     64     278   
2021-02-22          85     262      50   58    125      33     64     278   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-20           99       37  ...      26     11       27        7   
2021-02-21           99       37  ...      26     11       27        7   
2021-02-22           99       37  ...      26     11       27        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-20             34     9    12          0       26      5  
2021-02-21             34     9    12          0       26      5  
2021-02-22             34     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000090,0.011776,0.001470,0.002536,0.001025,0.091241,0.002893,0.001312,0.000207,0.004250,...,0.002545,0.015299,0.000427,0.0,0.003619,0.002540,0.003482,0.000000,0.009440,0.001624
2021-02-21,0.000432,0.011952,0.001369,0.002530,0.000976,0.000000,0.001800,0.000987,0.000138,0.004144,...,0.002388,0.012499,0.000276,0.0,0.003429,0.006334,0.007660,0.003709,0.008242,0.000783
2021-02-22,0.000234,0.010365,0.001582,0.001215,0.001413,0.026756,0.002624,0.000610,0.000242,0.002852,...,0.002582,0.009372,0.000339,0.0,0.003506,0.003777,0.007683,0.005081,0.007033,0.001844


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000000,0.010391,0.001354,0.000000,0.000000,0.100000,0.002392,0.000317,0.0,0.002396,...,0.003712,0.010657,0.0,NaN,0.003058,0.0,0.005099,0.000000,0.013972,0.001399
2021-02-21,0.000823,0.007864,0.001014,0.000000,0.002008,0.181818,0.001487,0.001583,0.0,0.002151,...,0.001799,0.008787,0.0,NaN,0.003049,0.0,0.002537,0.001618,0.003937,0.002793
2021-02-22,0.000411,0.009004,0.001013,0.018692,0.000000,0.076923,0.003145,0.000948,0.0,0.001312,...,0.001473,0.015679,0.0,NaN,0.003040,0.0,0.005061,0.000000,0.010784,0.003482


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000348,0.015064,0.001862,0.002365,0.000737,0.014019,0.003119,0.000698,0.000087,0.003900,...,0.003822,0.010639,0.000346,0.0,0.003579,0.000000,0.003690,0.0,0.014088,0.003878
2021-02-21,0.000287,0.012729,0.001768,0.003540,0.000421,0.004608,0.002723,0.000562,0.000044,0.002924,...,0.001594,0.008369,0.000320,0.0,0.003840,0.046097,0.004025,0.0,0.014953,0.000904
2021-02-22,0.001249,0.015617,0.001933,0.003821,0.009309,0.018349,0.003060,0.000642,0.000131,0.003096,...,0.002741,0.012536,0.000243,0.0,0.003833,0.008813,0.003810,0.0,0.014483,0.002833


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,0.001595,0.000000,NaN,0.002546,0.001643,0.001736,0.002668,0.000000,0.004158,0.0,...,0.001750,0.002359,0.002130,0.007578,0.000000,0.003361,0.002673,0.002394,0.001334,0.000000
2021-02-21,0.001763,0.000000,NaN,0.002238,0.000901,0.001278,0.001910,0.000000,0.002378,0.0,...,0.001478,0.002639,0.001915,0.009332,0.000000,0.004099,0.000000,0.001948,0.000737,0.002086
2021-02-22,0.001391,0.006011,NaN,0.001865,0.000777,0.000994,0.001617,0.008176,0.004225,0.0,...,0.001244,0.002251,0.000922,0.007935,0.001553,0.002047,0.003499,0.001836,0.000771,0.001952


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-20      0.002139  0.001601  0.002911  0.001244  0.003156  0.001744   
2021-02-21      0.004104  0.002114  0.001451  0.000829  0.000331  0.001741   
2021-02-22      0.000654  0.001441  0.001932  0.000414  0.000166  0.001738   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-20      0.002068  0.004277  0.001787  0.001142  ...  0.000000   
2021-02-21      0.001032  0.001936  0.000892  0.000000  ...  0.002703   
2021-02-22      0.002577  0.005951  0.000000  0.000000  ...  0.002310   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-02-20      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   
2021-02-21      0.000000  0.001339      0.0   0.004115  0.002411  0.001482   
2021-02-22      0.005172  0.001671      0.0   0.007104  0.001503  0.001480   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-20            NaN  0.000000  0.000000  
2021-02-21            NaN  0.000000  0.000000  
2021-02-22            NaN  0.002281  0.001616  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,0.001776,0.00000,NaN,0.003826,0.002249,0.005815,0.001531,0.000000,0.000000,NaN,...,0.004610,0.004996,0.004362,0.015544,0.0,0.013948,0.000000,0.002669,0.002773,0.0
2021-02-21,0.000209,0.00000,NaN,0.001615,0.001683,0.004581,0.000849,0.000000,0.000732,NaN,...,0.001619,0.002878,0.005429,0.005102,0.0,0.018619,0.000000,0.003106,0.000000,0.0
2021-02-22,0.000000,0.00346,NaN,-0.000193,0.001120,0.002635,0.000170,0.005184,0.000732,NaN,...,0.001796,0.003297,0.000540,0.005076,0.0,0.021143,0.007258,0.000885,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                        \
Admin2         Autauga Baldwin   Barbour Bibb Blount Bullock Butler  Calhoun   
2021-02-20         0.0     0.0  0.020408  0.0    0.0     0.0    0.0  0.00361   
2021-02-21         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.00000   
2021-02-22         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.00000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-02-21          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-02-22          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-20            0.0   0.0   0.0        NaN      0.0    0.0  
2021-02-21            0.0   0.0   0.0        NaN      0.0    0.0  
2021-02-22            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000204,0.011402,0.001541,0.002737,0.001235,0.063671,0.003195,0.001226,0.000161,0.004164,...,0.002745,0.013234,0.000500,0.0,0.003219,0.005438,0.003747,0.000689,0.010048,0.002741
2021-02-21,0.000318,0.011677,0.001455,0.002634,0.001106,0.031836,0.002498,0.001106,0.000149,0.004154,...,0.002567,0.012867,0.000388,0.0,0.003324,0.005886,0.005704,0.002199,0.009145,0.001762
2021-02-22,0.000276,0.011021,0.001518,0.001924,0.001259,0.029296,0.002561,0.000858,0.000196,0.003503,...,0.002575,0.011119,0.000363,0.0,0.003415,0.004831,0.006693,0.003640,0.008089,0.001803


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000126,0.010720,0.001247,0.000038,0.000899,0.057040,0.002848,0.000525,1.528391e-20,0.003049,...,0.004145,0.010082,1.572568e-06,NaN,0.003629,1.328708e-53,0.004067,0.000064,0.012174,0.003089
2021-02-21,0.000475,0.009292,0.001130,0.000019,0.001454,0.119429,0.002168,0.001054,7.641954e-21,0.002600,...,0.002972,0.009435,7.862838e-07,NaN,0.003339,6.643538e-54,0.003302,0.000841,0.008056,0.002941
2021-02-22,0.000443,0.009148,0.001072,0.009355,0.000727,0.098176,0.002656,0.001001,3.820977e-21,0.001956,...,0.002223,0.012557,3.931419e-07,NaN,0.003189,3.321769e-54,0.004181,0.000421,0.009420,0.003211


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-20,0.000859,0.015445,0.001960,0.003339,0.000993,0.010567,0.003193,0.000874,0.000128,0.003456,...,0.005440,0.010459,0.000458,0.0,0.003328,0.006720,0.004195,0.000112,0.013606,0.005719
2021-02-21,0.000573,0.014087,0.001864,0.003439,0.000707,0.007588,0.002958,0.000718,0.000086,0.003190,...,0.003517,0.009414,0.000389,0.0,0.003584,0.026409,0.004110,0.000056,0.014279,0.003311
2021-02-22,0.000911,0.014852,0.001898,0.003630,0.005008,0.012968,0.003009,0.000680,0.000108,0.003143,...,0.003129,0.010975,0.000316,0.0,0.003709,0.017611,0.003960,0.000028,0.014381,0.003072


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,0.001749,0.001750,NaN,0.002262,0.001336,0.001972,0.002759,0.001728,0.003786,7.599650e-104,...,0.001642,0.002059,0.002313,0.007796,0.002510,0.003596,0.002852,0.002394,0.001372,0.001156
2021-02-21,0.001756,0.000875,NaN,0.002250,0.001118,0.001625,0.002335,0.000864,0.003082,3.799825e-104,...,0.001560,0.002349,0.002114,0.008564,0.001255,0.003848,0.001426,0.002171,0.001054,0.001621
2021-02-22,0.001573,0.003443,NaN,0.002057,0.000947,0.001310,0.001976,0.004520,0.003653,1.899913e-104,...,0.001402,0.002300,0.001518,0.008249,0.001404,0.002948,0.002463,0.002004,0.000913,0.001787


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-20      0.002086  0.002081  0.002335  0.001618  0.002569  0.001644   
2021-02-21      0.003095  0.002097  0.001893  0.001223  0.001450  0.001692   
2021-02-22      0.001874  0.001769  0.001913  0.000819  0.000808  0.001715   

Province_State                                         ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers Cherokee  ...      Park   
2021-02-20      0.002548  0.003459  0.002199  0.00076  ...  0.000914   
2021-02-21      0.001790  0.002698  0.001545  0.00038  ...  0.001808   
2021-02-22      0.002184  0.004324  0.000773  0.00019  ...  0.002059   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-20      0.004670  0.000479  0.000411   0.002225  0.001407  0.001167   
2021-02-21      0.002335  0.000909  0.000206   0.003170  0.001909  0.001325   
2021-02-22      0.003754  0.001290  0.000103   0.005137  0.001706  0.001402   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-20           -1.0  0.000754  0.000506  
2021-02-21           -1.0  0.000377  0.000253  
2021-02-22           -1.0  0.001329  0.000934  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-20,0.006628,0.001655,NaN,0.006495,0.002443,0.007684,0.002080,0.001624,0.000221,NaN,...,0.003569,0.004424,0.005961,0.009351,0.000041,0.008008,0.003071,0.003678,0.003338,0.000734
2021-02-21,0.003418,0.000827,NaN,0.004055,0.002063,0.006132,0.001465,0.000812,0.000476,NaN,...,0.002594,0.003651,0.005695,0.007227,0.000020,0.013313,0.001535,0.003392,0.001669,0.000367
2021-02-22,0.001709,0.002144,NaN,0.001931,0.001591,0.004383,0.000817,0.002998,0.000604,NaN,...,0.002195,0.003474,0.003118,0.006151,0.000010,0.017228,0.004397,0.002138,0.000834,0.000183


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-20      0.003087  0.008625  0.011603  0.004504  0.008365  0.002018   
2021-02-21      0.001544  0.004313  0.005802  0.002252  0.004182  0.001009   
2021-02-22      0.000772  0.002156  0.002901  0.001126  0.002091  0.000505   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-20      0.000016  0.013911  0.005324  0.000007  ...  0.001272   
2021-02-21      0.000008  0.006956  0.002662  0.000003  ...  0.000636   
2021-02-22      0.000004  0.003478  0.001331  0.000002  ...  0.000318   

Province_State                                                             \
Admin2                Platte  Sheridan      Sublette Sweetwater     Teton   
2021-02-20      1.503052e-09  0.005446  1.228647e-15   0.000007  0.003922   
2021-02-21      7.515260e-10  0.002723  6.143234e-16   0.000004  0.001961   
2021-02-22      3.757630e-10  0.001362  3.071617e-16   0.000002  0.000981   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-02-20      1.993173e-10  -0.501961  7.633790e-08  1.621246e-06  
2021-02-21      9.965865e-11  -0.501961  3.816895e-08  8.106232e-07  
2021-02-22      4.982932e-11  -0.501961  1.908447e-08  4.053116e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210222,AK,55507,NaN,NaN,NaN,totalTestsViral,1648785,39.0,1251.0,...,NaN,1,8,a010200fddf461e02b94f41f860503a1277ad914,0,0,0,0,0,NaN
1,20210222,AL,487520,105189.0,1877930.0,NaN,totalTestsPeopleViral,2260261,862.0,45128.0,...,NaN,0,361,2bf6f3ebf4634a6a78de34bc4d359290bc7891a3,0,0,0,0,0,NaN
2,20210222,AR,315759,65912.0,2355211.0,NaN,totalTestsViral,2605058,588.0,14570.0,...,NaN,6,30,580dbd486272563eb9a1af92e7b3ce8342715780,0,0,0,0,0,NaN
3,20210222,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,53877ea68e8777cf77427f0e3b38715a519c5b61,0,0,0,0,0,NaN
4,20210222,AZ,809474,54423.0,2948102.0,NaN,totalTestsViral,7458884,1590.0,56994.0,...,NaN,-3,48,0edc09671f528e66c39e6df3b8bac67f93bb8771,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210222,AK,55507.0,NaN,NaN,NaN,totalTestsViral,1648785.0,39.0,1251.0,...,NaN,1,8,a010200fddf461e02b94f41f860503a1277ad914,0,0,0,0,0,NaN
1,20210222,AL,487520.0,105189.0,1877930.0,NaN,totalTestsPeopleViral,2260261.0,862.0,45128.0,...,NaN,0,361,2bf6f3ebf4634a6a78de34bc4d359290bc7891a3,0,0,0,0,0,NaN
2,20210222,AR,315759.0,65912.0,2355211.0,NaN,totalTestsViral,2605058.0,588.0,14570.0,...,NaN,6,30,580dbd486272563eb9a1af92e7b3ce8342715780,0,0,0,0,0,NaN
3,20210222,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,53877ea68e8777cf77427f0e3b38715a519c5b61,0,0,0,0,0,NaN
4,20210222,AZ,809474.0,54423.0,2948102.0,NaN,totalTestsViral,7458884.0,1590.0,56994.0,...,NaN,-3,48,0edc09671f528e66c39e6df3b8bac67f93bb8771,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-22,AK,55507,NaN,NaN,NaN,totalTestsViral,1648785,39.0,1251.0,NaN,...,NaN,1,8,a010200fddf461e02b94f41f860503a1277ad914,0,0,0,0,0,NaN
2021-02-22,AL,487520,105189.0,1877930.0,NaN,totalTestsPeopleViral,2260261,862.0,45128.0,NaN,...,NaN,0,361,2bf6f3ebf4634a6a78de34bc4d359290bc7891a3,0,0,0,0,0,NaN
2021-02-22,AR,315759,65912.0,2355211.0,NaN,totalTestsViral,2605058,588.0,14570.0,225.0,...,NaN,6,30,580dbd486272563eb9a1af92e7b3ce8342715780,0,0,0,0,0,NaN
2021-02-22,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,53877ea68e8777cf77427f0e3b38715a519c5b61,0,0,0,0,0,NaN
2021-02-22,AZ,809474,54423.0,2948102.0,NaN,totalTestsViral,7458884,1590.0,56994.0,478.0,...,NaN,-3,48,0edc09671f528e66c39e6df3b8bac67f93bb8771,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-22,AK,55507.0,NaN,NaN,NaN,totalTestsViral,1648785.0,39.0,1251.0,NaN,...,NaN,1,8,a010200fddf461e02b94f41f860503a1277ad914,0,0,0,0,0,NaN
2021-02-22,AL,487520.0,105189.0,1877930.0,NaN,totalTestsPeopleViral,2260261.0,862.0,45128.0,NaN,...,NaN,0,361,2bf6f3ebf4634a6a78de34bc4d359290bc7891a3,0,0,0,0,0,NaN
2021-02-22,AR,315759.0,65912.0,2355211.0,NaN,totalTestsViral,2605058.0,588.0,14570.0,225.0,...,NaN,6,30,580dbd486272563eb9a1af92e7b3ce8342715780,0,0,0,0,0,NaN
2021-02-22,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,53877ea68e8777cf77427f0e3b38715a519c5b61,0,0,0,0,0,NaN
2021-02-22,AZ,809474.0,54423.0,2948102.0,NaN,totalTestsViral,7458884.0,1590.0,56994.0,478.0,...,NaN,-3,48,0edc09671f528e66c39e6df3b8bac67f93bb8771,0,0,0,0,0,NaN
